In [11]:
import pandas as pd
import pywikibot 
import re
import requests

site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()

df = pd.read_csv('Coins.csv')
df['ID']=df['ID'].astype(str)
df['ObjectNumber']=df['ObjectNumber'].astype(str)

batch = df[2740:2825]

### FIND ITEMS ALREADY IN WIKIDATA
URL = "https://query.wikidata.org/sparql?query= SELECT ?item ?ID WHERE {?item wdt:P8583 ?ID . SERVICE wikibase:label { bd:serviceParam wikibase:language 'en' . }} &format = JSON"
headers = {"User" : "DuraEuroposBot/0.0 (https://www.wikidata.org/wiki/User_talk:Valeriummaximum) pywikibot_data_import",
           "Accept" : "application/json"
          }
read = requests.get(URL, headers=headers)
read=read.json()

items = []
ids = []
for item in read['results']['bindings']:
    val = item['item']['value']
    val2 = item['ID']['value']
    Qid = re.sub(r'.*/','',val)
    items.append(Qid)
    ids.append(val2)
d = {'Qid': items, 'ID':ids}
wikidata_items = pd.DataFrame(d)

In [12]:
wikidata_items.loc[wikidata_items['ID']=='147558','Qid']

9470    Q100584167
Name: Qid, dtype: object

In [13]:
for artifact in batch.itertuples(index=False):
    
    ### Wikidata property claims
    prop_ID= pywikibot.Claim(repo, u'P8583') 
    prop_Collection = pywikibot.Claim(repo, u'P195')
    Inventory_qual = pywikibot.Claim(repo, u'P217')
    prop_Settlement = pywikibot.Claim(repo, u'P276')
    
    prop_Inventory = pywikibot.Claim(repo, u'P217')
    collection_qual = pywikibot.Claim(repo, u'P195')
    YUAG = pywikibot.ItemPage(repo, u"Q1568434")
    collection_qual.setTarget(YUAG)

    prop_Date = pywikibot.Claim(repo, u'P571')
    EarliestDate_qual = pywikibot.Claim(repo, u'P1319')
    LatestDate_qual = pywikibot.Claim(repo, u'P1326')
    Circa_qual = pywikibot.Claim(repo,u"P1480")

    prop_Location = pywikibot.Claim(repo, u'P189')
    prop_Instance = pywikibot.Claim(repo, u'P31')
    prop_coin = pywikibot.Claim(repo, u'P31')

    prop_Commons = pywikibot.Claim(repo, u'P4765')
    License_qual = pywikibot.Claim(repo, u'P275')
    Format_qual = pywikibot.Claim(repo, u'P2701')
    Title_qual = pywikibot.Claim(repo,u'P1476')
    
    prop_copyright = pywikibot.Claim(repo, u'P6216')
    public_domain = pywikibot.ItemPage(repo, u"Q19652")
    prop_copyright.setTarget(public_domain)
    
    prop_creator = pywikibot.Claim(repo,u'P170')
    anonymous = pywikibot.ItemPage(repo, u"Q4233718")
    prop_creator.setTarget(anonymous)
    
    prop_weight = pywikibot.Claim(repo,u'P2067')
    prop_diameter = pywikibot.Claim(repo,u'P2386')
    
    ### ADD URL FOR EACH CLAIM
    url_qual = pywikibot.Claim(repo, u'P854')
    prop_ID.addSource(url_qual)
    prop_Date.addSource(url_qual)
    prop_Collection.addSource(url_qual)
    prop_Location.addSource(url_qual)
    prop_Commons.addSource(url_qual)
    prop_Inventory.addSource(url_qual)
    prop_diameter.addSource(url_qual)
    prop_copyright.addSource(url_qual)
    prop_creator.addSource(url_qual)
    prop_weight.addSource(url_qual)
    
    def weight(x):
        if pd.isna(x.weight)==False:
            unit_gram = pywikibot.ItemPage(repo, "Q41803")
            value = float(x.weight)
            weight = pywikibot.WbQuantity(value,unit_gram)
            prop_weight.setTarget(weight)
            claims.append(prop_weight)
    
    def diameter(x):
        if pd.isna(x.diameter)==False:
            unit_mm = pywikibot.ItemPage(repo, "Q174789")
            value = float(x.diameter)
            diameter = pywikibot.WbQuantity(value,unit_mm)
            prop_diameter.setTarget(diameter)
            claims.append(prop_diameter)
    
    def date_normalise(x):
        if pd.isna(x.DateCertainty)==False:
            if 'circa' in x.DateCertainty:
                circa = pywikibot.ItemPage(repo, u"Q5727902")
                Circa_qual.setTarget(circa)
                prop_Date.addQualifier(Circa_qual)
            if 'Possibly' in x.DateCertainty:
                possibly = pywikibot.ItemPage(repo, u"Q30230067")
                Circa_qual.setTarget(possibly)
                prop_Date.addQualifier(Circa_qual)
        
        if pd.isna(x.EarliestDate)==False:
            val = x.EarliestDate[0:5]
            val = re.sub(r'(?<=\+|\-)(0)*?(?=[1-9])','',val)
            val=int(val)
            target = pywikibot.WbTime(year=val)
            EarliestDate_qual.setTarget(target)
            prop_Date.addQualifier(EarliestDate_qual)
        if pd.isna(x.LatestDate)==False:
            val = x.LatestDate[0:5]
            val = re.sub(r'(?<=\+|\-)(0)*?(?=[1-9])','',val)
            val=int(val)
            target = pywikibot.WbTime(year=val)
            LatestDate_qual.setTarget(target)
            prop_Date.addQualifier(LatestDate_qual)
    
        if x.Date=='somevalue':
             prop_Date.setSnakType('somevalue')
             claims.append(prop_Date)
        else:
            if 'Z/9' in x.Date:
                val = x.Date[0:5]
                val = re.sub(r'(?<=\+|\-)(0)*?(?=[1-9])','',val)
                val=int(val)
                target = pywikibot.WbTime(year=val)
                prop_Date.setTarget(target)
                claims.append(prop_Date)
            if 'Z/7' in x.Date:
                val = x.Date[0:5]
                val = re.sub(r'(?<=\+|\-)(0)*?(?=[1-9])','',val)
                val=int(val)
                target = pywikibot.WbTime(year=val, precision=7)
                prop_Date.setTarget(target)
                claims.append(prop_Date)  
    
    ###Check Items in Wikidata, update items, create items, create claims, iterate over addClaims, iterate over rows
    if str(artifact.ID) in ids:
        claims=[]
        val = str(artifact.ID)
        Qid = wikidata_items.loc[wikidata_items.ID==val, 'Qid'].item()
        item = pywikibot.ItemPage(repo, Qid)
        
        #SET URL REFERENCE
        url_qual.setTarget(artifact.Reference) 
               
        ###ADD Dates, SOMEVALUE, CENTURY, YEAR, QUALIFIERS (if not null)  
        if 'P571' not in item.get()['claims'].keys():
            date_normalise(artifact)
    
        ###Add Place of Excavation if not already there
        if 'P189' not in item.get()['claims'].keys():
            Dura = pywikibot.ItemPage(repo, u"Q464266")
            prop_Location.setTarget(Dura)
            claims.append(prop_Location)
        
        ### Add settlement of object if not there
        if 'P276' not in item.get()['claims'].keys():
            New_Haven = pywikibot.ItemPage(repo, u'Q49145')
            prop_Settlement.setTarget(New_Haven)
            claims.append(prop_Settlement)
        
        ### Add dimensions 
        if 'P2067' not in item.get()['claims'].keys():
            weight(artifact)                
        if 'P2386' not in item.get()['claims'].keys():
            diameter(artifact)
        
        #Check Collection Inventory Number
        ObjNo = artifact.ObjectNumber
        ObjNo = re.sub('inv\. ', '', ObjNo)
        if 'P195' in item.get()['claims'].keys():
            for claim in item.claims['P195']:            
                if 'P217' not in claim.qualifiers: 
                    Inventory_qual.setTarget(ObjNo)
                    prop_Collection.addQualifier(Inventory_qual)
        else:
            YaleGallery = pywikibot.ItemPage(repo, u"Q1568434") 
            prop_Collection.setTarget(YaleGallery)
            Inventory_qual.setTarget(ObjNo)
            prop_Collection.addQualifier(Inventory_qual)
            claims.append(prop_Collection)
        
        #ADD Inventory
        if 'P217' not in item.get()['claims'].keys():
            prop_Inventory.setTarget(ObjNo)
            prop_Inventory.addQualifier(collection_qual)
            claims.append(prop_Inventory)
        
        #ADD IMAGE URL
        if 'P4765' not in item.get()['claims'].keys():
            if pd.isna(artifact.url)==False:
                prop_Commons.setTarget(artifact.url)
                CC0 = pywikibot.ItemPage(repo, u"Q6938433")
                License_qual.setTarget(CC0)
                prop_Commons.addQualifier(License_qual)
                title_target = pywikibot.WbMonolingualText(artifact.Title, 'en')
                Title_qual.setTarget(title_target)
                prop_Commons.addQualifier(Title_qual)
                if 'tif' in artifact.formatresource:
                    Format_qual.setTarget(pywikibot.ItemPage(repo, u"Q215106"))
                    prop_Commons.addQualifier(Format_qual)
                elif 'jpg' in artifact.formatresource:
                    Format_qual.setTarget(pywikibot.ItemPage(repo, u"Q2195"))
                    prop_Commons.addQualifier(Format_qual)
                claims.append(prop_Commons)
    
        #If not described as archaeological object, add this statement
        if 'P31' in item.get()['claims']:
            lst=[]
            for clm in item.get()['claims']['P31']:
                target = clm.getTarget()
                target=str(target)
                lst.append(target)
            if '[[wikidata:Q220659]]' not in lst:
                archaeological_artifact = pywikibot.ItemPage(repo, u"Q220659")                
                prop_Instance.setTarget(archaeological_artifact)
                prop_Instance.addSource(url_qual)
                claims.append(prop_Instance)             
            if '[[wikidata:Q41207]]' not in lst: 
                coin = pywikibot.ItemPage(repo, u"Q41207")
                prop_coin.setTarget(coin)
                prop_coin.addSource(url_qual)
                claims.append(prop_coin)   
        else:
            archaeological_artifact = pywikibot.ItemPage(repo, u"Q220659")                
            prop_Instance.setTarget(archaeological_artifact)
            prop_Instance.addSource(url_qual)
            claims.append(prop_Instance)
            coin = pywikibot.ItemPage(repo, u"Q41207")
            prop_coin.setTarget(coin)
            prop_coin.addSource(url_qual)
            claims.append(prop_coin)  
        
        if 'P6216' not in item.get()['claims'].keys():
            claims.append(prop_copyright) 
    
        if 'P170' not in item.get()['claims'].keys(): 
            claims.append(prop_creator)
        
        #Update labels to include unique inventory numbers and IDs
        label = artifact.Title
        descriptions={'en': "YUAG " + artifact.ID + ". Archaeological artifact excavated in Dura Europos by the Yale-French team, 1928-1937, Syria"}
        
        if item.get()['labels'].get('en') != label:
            item.editLabels(labels={'en': label }, summary='Synchronizing Wikidata entry with Yale gallery records for ' + artifact.ObjectNumber)
            item.editDescriptions(descriptions, summary='Synchronizing Wikidata entry with Yale gallery records for ' + artifact.ObjectNumber)
        
        for stmnt in claims:
            item.addClaim(stmnt,summary='Importing Yale gallery records for ' + artifact.ObjectNumber)
    
    
    ### else ---> CREATE NEW ITEM:
    
    else:
        claims=[]
        item = pywikibot.ItemPage(repo)
        item.editEntity(artifact)
        url_qual.setTarget(artifact.Reference)
        
        ###ADD YUAG ID
        prop_ID.setTarget(artifact.ID)
        claims.append(prop_ID)
        
        ###Add Location of discovery
        Dura = pywikibot.ItemPage(repo, u"Q464266")
        prop_Location.setTarget(Dura)
        claims.append(prop_Location)
        
        ###Add Collection and Inventory Number Qualifier
        ObjNo = artifact.ObjectNumber
        ObjNo = re.sub('inv\. ', '', ObjNo)
        YaleGallery = pywikibot.ItemPage(repo, u"Q1568434") 
        prop_Collection.setTarget(YaleGallery)
        Inventory_qual.setTarget(ObjNo)
        prop_Collection.addQualifier(Inventory_qual)
        claims.append(prop_Collection)
        
        prop_Inventory.setTarget(ObjNo)
        prop_Inventory.addQualifier(collection_qual)
        claims.append(prop_Inventory)
            
        #Add New Haven as Settlement of Object
        New_Haven = pywikibot.ItemPage(repo, u'Q49145')
        prop_Settlement.setTarget(New_Haven)
        claims.append(prop_Settlement)
    
        ###Add Instance of Archaeological Artifact
        archaeological_artifact = pywikibot.ItemPage(repo, u"Q220659")
        prop_Instance.setTarget(archaeological_artifact)
        prop_Instance.addSource(url_qual)
        claims.append(prop_Instance)
        
        ###Instance of Coin
        coin = pywikibot.ItemPage(repo, u"Q41207")
        prop_coin.setTarget(coin)
        prop_coin.addSource(url_qual)
        claims.append(prop_coin)
        
        ###ADD Dates, SOMEVALUE, CENTURY, YEAR, QUALIFIERS (if not null)
        date_normalise(artifact)
        
        ### Add dimensions 
        weight(artifact)
        diameter(artifact)
        
        #ADD IMAGE URL
        if pd.isna(artifact.url)==False:
            prop_Commons.setTarget(artifact.url)
            CC0 = pywikibot.ItemPage(repo, u"Q6938433")
            License_qual.setTarget(CC0)
            prop_Commons.addQualifier(License_qual)
            title_target = pywikibot.WbMonolingualText(artifact.Title, 'en')
            Title_qual.setTarget(title_target)
            prop_Commons.addQualifier(Title_qual)
            if 'tif' in artifact.formatresource:
                Format_qual.setTarget(pywikibot.ItemPage(repo, u"Q215106"))
                prop_Commons.addQualifier(Format_qual)
            elif 'jpg' in artifact.formatresource:
                Format_qual.setTarget(pywikibot.ItemPage(repo, u"Q2195"))
                prop_Commons.addQualifier(Format_qual)
            claims.append(prop_Commons)
        
        claims.append(prop_copyright) 
        claims.append(prop_creator)
        
        ###Add labels and descriptions
        new_label = {"en": artifact.Title}
        new_description = {"en": "YUAG " + artifact.ID + ". Archaeological object excavated in Dura Europos by the Yale-French team, 1928-1937, Syria"}

        item.editLabels(labels=new_label, summary='Importing Yale gallery records for ' + artifact.ObjectNumber)
        item.editDescriptions(descriptions=new_description, summary='Importing Yale gallery records for ' + artifact.ObjectNumber)
        
        #add Claims
        for stmnt in claims:
            item.addClaim(stmnt,summary='Importing Yale gallery records for ' + artifact.ObjectNumber)